In [31]:
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn

# Quick NN modeling:

In [25]:
X = df[inputs.keys()]
y = df[outputs.keys()]

In [35]:
# Assume X and y are your multiple-target DataFrame/array
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale inputs and outputs
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
y_test_tensor = torch.FloatTensor(y_test_scaled)

# Neural Network
class MultiTargetNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, output_dim)
        )
    
    def forward(self, x):
        return self.layers(x)

# Model initialization
model = MultiTargetNN(X_train_scaled.shape[1], y_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Predictions and R2 Score
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).numpy()
    y_pred = y_scaler.inverse_transform(y_pred_scaled)
    
r2_scores = [r2_score(y_test.iloc[:, i], y_pred[:, i]) for i in range(y_test.shape[1])]
print("R2 Scores:", r2_scores)

R2 Scores: [0.9399873843860569, 0.8152910574243667, 0.9075037158967714, 0.8665251563907919]


### Save dataset

In [19]:
filename = "laser_welding_dataset.pkl"
dataset_to_save = df_scaled

In [20]:
filepath = os.path.join(os.getcwd(), "../", "datasets", filename)
with open(filepath, "wb") as handle:
    pickle.dump(dataset_to_save, handle)

### Load dataset

In [21]:
with open(filepath, "rb") as f:
    dataset = pickle.load(f)

In [22]:
dataset

,Laser Power_W,Pulse Duration_ms,Welding Speed_mm/s,Beam Diameter_mm,Focal Position_mm,Flow Rate_L/min,Heat Input_J/mm,Ambient Temperature_degC,Cooling Rate_degC/s,Hardness_HV,Fatigue Life_numCycles,Wear Rate_mg/m,Cutting Efficiency_m/s
0,113.713642,2.679319,120.935883,2.096005,-1.482082,14.718795,126.113219,29.970587,294.999900,286.968224,76585.547661,0.766614,1.423620
1,772.844484,9.004940,10.439518,1.857986,4.501493,23.237073,397.839005,21.596130,797.461569,517.568430,90136.271416,0.118067,2.928525
2,918.927783,5.466613,7.111312,2.757836,4.897038,24.606440,261.280991,22.318435,644.429357,339.132412,93376.078194,0.051226,4.358287
3,283.048002,2.941247,147.348596,0.838138,-0.349865,15.791748,417.829625,27.401784,902.487845,752.795608,29779.898934,0.958867,0.582143
4,366.232143,6.082522,185.396807,1.533911,0.979201,18.764714,498.641809,20.670720,555.742623,737.530363,21945.819010,0.475914,0.180458
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,561.709730,8.346749,28.087096,1.474552,3.563197,24.539299,73.252267,26.125094,407.775649,218.017766,98499.355157,0.407204,4.353199
76,717.699228,3.963958,158.491867,1.502677,3.690833,9.858366,223.177980,24.912593,819.901226,753.813389,39013.626029,0.699564,4.360952
77,789.270262,2.338064,15.942385,0.824970,0.532259,23.306000,342.618033,24.713617,67.733543,238.947482,52966.757203,0.271263,2.181592
78,391.156012,3.725438,99.687214,2.472764,4.029180,6.987129,335.845966,23.084795,148.168380,749.169491,68584.048989,0.029296,3.448862
